In [1]:
import warnings

warnings.filterwarnings("ignore", message=".*Failed to load image Python extension.*")

from mlp import MLP
from functions import *

train_dataset, test_dataset = load_mnist()  # Load the MNIST dataset
img, label = train_dataset[0]  # Get the first image and its label

In [2]:
# Process the image 
test_img = process_image(img)
test_label = one_hot_encoder(label)
# Create the MLP
mlp = MLP(activation_function="relu")
# Perform a forward pass
output=mlp.forward(test_img)
loss = mean_squared_error(output, test_label)

# print("Output:", output.shape)
# print("label:", test_label.shape)
print(loss)

0.1999603125860326
